<a href="https://colab.research.google.com/github/chopstickexe/deep-learning-from-scratch-2/blob/master/ch06_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 下準備

$$
\newcommand{\vect}[1]{\mathbf{#1}}
\newcommand{\mat}[1]{\mathbf{#1}}
$$

## 数式の表記

変数: 小文字イタリック $x$

定数: 大文字イタリック $X$

ベクトル: 小文字ローマン体太字 $\vect{x}$

行列: 大文字ローマン体太字 $\mat{X}$

## 公式実装のclone

In [1]:
!git clone --depth=1 https://github.com/oreilly-japan/deep-learning-from-scratch-2.git
import sys 
sys.path.append('deep-learning-from-scratch-2')

Cloning into 'deep-learning-from-scratch-2'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 73 (delta 13), reused 14 (delta 0), pack-reused 0
Unpacking objects: 100% (73/73), done.


# 6章 ゲート付きRNN

## 6.1 RNNの問題点（勾配消失・勾配爆発）

5章のシンプルなRNNは時系列が長くなると，逆伝播の際にtanhの微分値

$$ 
\frac{\delta \mathrm{tanh}(\vect{x})}{\delta \vect{x}} = 1 - \mathrm{tanh}(\vect{x})^2, (-1 \leq \mathrm{tanh}(\vect{x}) \leq 1)
$$

をtごとに損失値の勾配にかけ算することで勾配消失につながりやすくなる．

また，同じ重み $\mat{W_h}$ を損失値の勾配に何度もかけ算することで，勾配爆発につながりやすくなる．

この勾配消失と勾配爆発の二つの問題のうち，勾配爆発は**勾配クリッピング**という比較的簡単な手法で抑えることができる．

勾配クリッピングは以下のように計算する．あるイテレーションの結果，ニューラルネットワークのすべてのパラメータの勾配（いままでの実装で言うところの`self.grads`）を$\hat{\vect{g}}$としたとき，そのL2ノルム$||\hat{\vect{g}}||$があらかじめ定められた閾値$\mathit{threshold}$を超えていたら，

$$
\hat{\vect{g}} = \frac{\mathit{threshold}}{||\hat{\vect{g}}||} \hat{\vect{g}}
$$

でパラメータの勾配を修正する．（勾配は必ず1倍以下の値になる）



## 6.2 勾配消失とLSTM

シンプルなRNNのもう一つの問題である勾配消失については以下のように解決する（これがいわゆるLSTM）．

まず，$\vect{h_{t-1}} \mat{W_h} + \vect{x_t} \mat{W_x} + \vect{b}$を必ずtanhに放り込むのではなく，別途直接次の時刻$\mathrm{t+1}$に渡すルートを作る．（勾配消失の原因になるtanhをスキップ）
このルートに相当する変数を**セル**（または記憶セル）と呼び，$\vect{c_t}$で表現する．

そして，以下の3種類のゲートを持たせる．

- Outputゲート（$\vect{o}$）: 出力する隠れベクトル$\vect{h_t}$にどのくらい$\tanh(\vect{c_t})$を反映させるか決める
- Forgetゲート（$\vect{f}$）: $\vect{c_{t-1}}$ をどのくらい $\vect{c_t}$ に反映させるか決める
- Inputゲート（$\vect{i}$）: 従来のシンプルなRNNの出力$\vect{g} = \tanh(\vect{h_{t-1}} \mat{W_h} + \vect{x_t} \mat{W_x} + \vect{b})$をどのくらい $\vect{c_t}$ に反映させるか決める．

これらのゲートの出力は0～1の係数になる．その値は，シンプルなRNNの出力と同様に，$\vect{h_{t-1}}$と$\vect{x_t}$から求める．具体的には以下のように，sigmoid関数（$\sigma$）を用いて求める．（$\otimes$はアダマール積を表し，ベクトルを要素ごとに積算する）

$$
\begin{align}
\vect{f} &= \sigma(\vect{h_{t-1}} \mat{W_h^{(\vect{f})}} + \vect{x_t} \mat{W_x^{(\vect{f})}} + \vect{b}) \\
\vect{g} &= \tanh(\vect{h_{t-1}} \mat{W_h^{(\vect{g})}} + \vect{x_t} \mat{W_x^{(\vect{g})}} + \vect{b}) \\
\vect{i} &= \sigma(\vect{h_{t-1}} \mat{W_h^{(\vect{i})}} + \vect{x_t} \mat{W_x^{(\vect{i})}} + \vect{b}) \\
\vect{o} &= \sigma(\vect{h_{t-1}} \mat{W_h^{(\vect{o})}} + \vect{x_t} \mat{W_x^{(\vect{o})}} + \vect{b}) \\ \\
\vect{c_t} &= \vect{f} \otimes \vect{c_{t-1}} + \vect{i} \otimes \vect{g} \\
\vect{h_t} &= \vect{o} \otimes \tanh(\vect{c_t})  
\end{align}
$$


## 6.3 LSTMの実装（`LSTM`，`TimeLSTM`クラス）

同じような行列積を4回計算することになるので，実装上は全ての$\mat{W_x}, \mat{W_h}, \vect{b}$を列方向にコンカチして，一回の行列積で4つの式の行列積を計算する．

In [2]:
class LSTM:
  def __init__(self, Wx, Wh, b):  # Wx, Wh, bはすでに4つ分コンカチ済みの想定
    self.params = [Wx, Wh, b]
    self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
    self.cache = None

  def forward(self, x, h_prev, c_prev):
    Wx, Wh, b = self.params
    N, H = h_prev.shape

    A = np.dot(x, Wx) + np.dot(h_prev, Wh) + b  # まとめて計算する(AはALLのA）

    # まとめて計算した結果をsliceで取得
    f = A[:, :H]
    g = A[:, H:2*H]
    i = A[:, 2*H:3*H]
    o = A[:, 3*H:]

    f = sigmoid(f)
    g = np.tanh(g)
    i = sigmoid(i)
    o = sigmoid(o)

    c_next = f * c_prev + g * i
    h_next = o * np.tanh(c_next)

    self.cache = (x, h_prev, c_prev, i, f, g, o, c_next)
    return h_next, c_next

TimeRNNと同様にTimeLSTMレイヤを実装する．内容はTimeRNNとほぼ同じなので省略．違いはRNNクラスの代わりにLSTMクラスを用いるのと，逆伝播時にdhとともにdcを渡してあげることだけ．

## 6.4 LSTMを使った言語モデル（`Rnnlm`クラス）

5章の`SimpleRnnlm`クラスのTimeRNN部分をTimeLSTMに変更するなどして，`Rnnlm`クラスを実装する．

これを`RnnlmTrainer`のモデルとして使って学習．（勾配クリッピングも利用）

In [3]:
%set_env PYTHONPATH=/content/deep-learning-from-scratch-2:/env/python
!echo $PYTHONPATH
!python /content/deep-learning-from-scratch-2/ch06/train_rnnlm.py

env: PYTHONPATH=/content/deep-learning-from-scratch-2:/env/python
/content/deep-learning-from-scratch-2:/env/python
Done
Done
| epoch 1 |  iter 1 / 1327 | time 0[s] | perplexity 10001.31
| epoch 1 |  iter 21 / 1327 | time 8[s] | perplexity 2666.52
| epoch 1 |  iter 41 / 1327 | time 16[s] | perplexity 1245.54
| epoch 1 |  iter 61 / 1327 | time 23[s] | perplexity 992.81
| epoch 1 |  iter 81 / 1327 | time 31[s] | perplexity 826.11
| epoch 1 |  iter 101 / 1327 | time 39[s] | perplexity 644.00
| epoch 1 |  iter 121 / 1327 | time 46[s] | perplexity 646.03
| epoch 1 |  iter 141 / 1327 | time 54[s] | perplexity 605.71
| epoch 1 |  iter 161 / 1327 | time 62[s] | perplexity 575.70
| epoch 1 |  iter 181 / 1327 | time 69[s] | perplexity 593.42
| epoch 1 |  iter 201 / 1327 | time 77[s] | perplexity 503.02
| epoch 1 |  iter 221 / 1327 | time 85[s] | perplexity 482.71
| epoch 1 |  iter 241 / 1327 | time 92[s] | perplexity 442.35
| epoch 1 |  iter 261 / 1327 | time 100[s] | perplexity 464.36
| epoch 1